In [4]:
import os

import cv2
import numpy as np
import pandas as pd
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizer_v2.gradient_descent import SGD
from tensorflow.python.layers.core import Flatten

from src.helpers import config


In [5]:
image_size = (224, 224)
image_size_ext = (224, 224, 3)
batch_size = 5
data_dir = "../data"


In [6]:

def get_random_images(count: int):
    images = []
    labels = []

    train_data_dir = "../../data/"
    for sub_dir in os.listdir(train_data_dir):
        image_list = os.listdir(
            os.path.join(train_data_dir, sub_dir)
        )  #list of all image names in the directory
        image_list = list(map(lambda x: os.path.join(sub_dir, x), image_list))
        images.extend(image_list)
        labels.extend([sub_dir] * len(image_list))

    df = pd.DataFrame({"Images": images, "Labels": labels})
    df = df.sample(frac=1).reset_index(drop=True)  # To shuffle the data
    df = df.head(count)  # to take the subset of data (I'm taking 100 from it)

    aug = ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        validation_slip=0.2,
    )

    train_generator = aug.flow_from_dataframe(
        dataframe=df,
        directory=train_data_dir,
        x_col="Images",
        y_col="Labels",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=image_size,
        subset="training"
    )

    validation_generator = aug.flow_from_dataframe(
        dataframe=df,
        directory=train_data_dir,
        x_col="Images",
        y_col="Labels",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=image_size,
        subset="validation"
    )
    return train_generator, validation_generator


In [7]:
def something(image_paths):
    data, labels = [], []
    for imagePath in image_paths:
        # extract the class label
        label = imagePath.split(os.path.sep)[-2]
        # load the image, convert it to RGB channel ordering, and resize
        # it to be a fixed 224x224 pixels, ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)
    # convert the data and labels to NumPy arrays
    print("[INFO] processing data...")
    data = np.array(data, dtype="float32")
    labels = np.array(labels)

    # perform one-hot encoding on the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    (trainX, testX, trainY, testY) = train_test_split(
        data, labels,
        test_size=0.2, random_state=42
    )
    # take the validation split from the training split
    (trainX, valX, trainY, valY) = train_test_split(
        trainX, trainY,
        test_size=0.2, random_state=84
    )
    # initialize the training data augmentation object
    aug = ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    return aug


In [8]:
def create_models():
    baseModel = VGG16(
        weights="imagenet", include_top=False,
        input_tensor=Input(shape=(224, 224, 3))
    )
    # construct the head of the model that will be placed on top of the
    # the base model
    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)
    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    model = Model(inputs=baseModel.input, outputs=headModel)
    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable
    print("[INFO] compiling model...")
    opt = SGD(lr=config.MIN_LR, momentum=0.9)
    model.compile(
        loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"]
    )

    return model

In [9]:
def fit(model, trainX, trainY, valX, valY, aug):
    H = model.fit_generator(
        aug.flow(trainX, trainY, batch_size=config.BATCH_SIZE),
        validation_data=(valX, valY),
        steps_per_epoch=trainX.shape[0] // config.BATCH_SIZE,
        epochs=config.NUM_EPOCHS,
        verbose=1
    )

    return H
